<a href="https://colab.research.google.com/github/LaAbadIAdelCrimen/abadia-3d/blob/master/abadia_3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation
import pandas as pd
from sys import exit


# Reading the data from a CSV file using pandas
repo = pd.read_csv('alturas-extra.csv',sep=',',header=0)
print(repo)


data = np.array((repo['X'].values, repo['Y'].values, repo['height'].values, repo['Level'].values))
print(data.shape[1])
# Create heatmap

data0 = np.asarray([x for x in data if x[3] == 0 ])
print(data0.shape)

heatmap, xedges, yedges = np.histogram2d(data[1], data[0], bins=(256,256))
extent = [xedges[0], xedges[256], yedges[0], yedges[256]]

# Plot heatmap
plt.clf()
plt.title('Plano de Alturas de la Abadia del Crimen')
plt.ylabel('x')
plt.xlabel('y')
plt.imshow(heatmap, extent=extent)
plt.show()

In [0]:
maps = np.zeros((3, 256, 256))
for dd in repo.values:
  # print (dd)
  maps[dd[0], dd[2], dd[1]] = dd[3]
  # print (maps[dd[0], dd[1], dd[2]])

# print (maps[2])

fig  = plt.figure(figsize=(10,10),facecolor = 'white', dpi=100)
plt.savefig('abadia-3d-planta0.png', dpi=100)

plt.imshow (maps[0][0:200], cmap='Blues', interpolation='none')
plt.show(fig)

fig  = plt.figure(figsize=(10,10),facecolor = 'white', dpi=100)
plt.savefig('abadia-3d-planta1.png', dpi=100)

plt.imshow (maps[1][0:120][0:120], cmap='Blues', interpolation='none')
plt.show(fig)

fig  = plt.figure(figsize=(10,10),facecolor = 'white', dpi=100)
plt.savefig('abadia-3d-planta2.png', dpi=100)

plt.imshow (maps[2][0:125][0:125], cmap='Blues', interpolation='none')
plt.show(fig)


In [0]:
import seaborn as sns
import matplotlib.pylab as plt
dd = np.random.rand(10, 12)
print(maps[2][0:125][0:125])
dd = maps[2][:150][:200]
fig  = plt.figure(figsize=(10,10),facecolor = 'white', dpi=100)
ax = sns.heatmap(dd, linewidth=0)
plt.show()

In [0]:
!pip install numpy-stl


In [0]:

import math
import stl
from stl import mesh
import numpy


# find the max dimensions, so we can know the bounding box, getting the height,
# width, length (because these are the step size)...
def find_mins_maxs(obj):
    minx = maxx = miny = maxy = minz = maxz = None
    for p in obj.points:
        # p contains (x, y, z)
        if minx is None:
            minx = p[stl.Dimension.X]
            maxx = p[stl.Dimension.X]
            miny = p[stl.Dimension.Y]
            maxy = p[stl.Dimension.Y]
            minz = p[stl.Dimension.Z]
            maxz = p[stl.Dimension.Z]
        else:
            maxx = max(p[stl.Dimension.X], maxx)
            minx = min(p[stl.Dimension.X], minx)
            maxy = max(p[stl.Dimension.Y], maxy)
            miny = min(p[stl.Dimension.Y], miny)
            maxz = max(p[stl.Dimension.Z], maxz)
            minz = min(p[stl.Dimension.Z], minz)
    return minx, maxx, miny, maxy, minz, maxz


def translate(_solid, step, padding, multiplier, axis):
    if axis == 'x':
        items = [0, 3, 6]
    elif axis == 'y':
        items = [1, 4, 7]
    elif axis == 'z':
        items = [2, 5, 8]
    for p in _solid.points:
        # point items are ((x, y, z), (x, y, z), (x, y, z))
        for i in range(3):
            p[items[i]] += (step * multiplier) + (padding * multiplier)


def copy_obj(obj, dims, num_rows, num_cols, num_layers):
    w, l, h = dims
    copies = []
    for layer in range(num_layers):
        for row in range(num_rows):
            for col in range(num_cols):
                # skip the position where original being copied is
                if row == 0 and col == 0 and layer == 0:
                    continue
                _copy = mesh.Mesh(obj.data.copy())
                # pad the space between objects by 10% of the dimension being
                # translated
                if col != 0:
                    translate(_copy, w, w / 10., col, 'x')
                if row != 0:
                    translate(_copy, l, l / 10., row, 'y')
                if layer != 0:
                    translate(_copy, h, h / 10., layer, 'z')
                copies.append(_copy)
    return copies

# Using an existing stl file:
main_body = mesh.Mesh.from_file('cube.stl')

# Dont want to rotate along Y
# main_body.rotate([0.0, 0.5, 0.0], math.radians(90))

minx, maxx, miny, maxy, minz, maxz = find_mins_maxs(main_body)
w1 = maxx - minx
l1 = maxy - miny
h1 = maxz - minz
copies = copy_obj(main_body, (w1, l1, h1), 1, 1, 1)

# I wanted to add another related STL to the final STL
twist_lock = mesh.Mesh.from_file('cube.stl')

minx, maxx, miny, maxy, minz, maxz = find_mins_maxs(twist_lock)
w2 = maxx - minx
l2 = maxy - miny
h2 = maxz - minz


translate(twist_lock, w1, w1 / 10., 3, 'x')

copies2 = copy_obj(twist_lock, (w2, l2, h2), 2, 2, 2)
combined = mesh.Mesh(numpy.concatenate([main_body.data, twist_lock.data] +
                                    [copy.data for copy in copies] +
                                    [copy.data for copy in copies2]))

combined.save('combined.stl', mode=stl.Mode.ASCII)  # save as ASCII


In [0]:
# Optionally render the rotated cube faces
from matplotlib import pyplot
from mpl_toolkits import mplot3d

# Create a new plot
figure = pyplot.figure()
axes = mplot3d.Axes3D(figure)

# Render the cube
axes.add_collection3d(mplot3d.art3d.Poly3DCollection(combined.vectors))

# Auto scale to the mesh size
scale = combined.points.flatten('C')
axes.auto_scale_xyz(scale, scale, scale)

# Show the plot to the screen
pyplot.show()

In [0]:


for ii in range(0,3):
  level = mesh.Mesh.from_file('cube.stl')

  copies = []
  for keys, val in np.ndenumerate(maps[ii]):
    if (val > 0): 
      _copy = mesh.Mesh(level.data.copy())
      translate(_copy, 1, 1 / 10., keys[0], 'x')
      translate(_copy, 1, 1 / 10., keys[1], 'y')
      translate(_copy, 1, 1 / 10., val, 'z')
      copies.append(_copy)

  combined2 = mesh.Mesh(numpy.concatenate([copy.data for copy in copies]))
  combined2.save(f"level{ii}.stl", mode=stl.Mode.ASCII)
  


In [0]:
# Create a new plot
figure = pyplot.figure()
axes = mplot3d.Axes3D(figure)

# Render the cube
axes.add_collection3d(mplot3d.art3d.Poly3DCollection(combined2.vectors))

# Auto scale to the mesh size
scale = combined2.points.flatten('C')
print(scale)
axes.auto_scale_xyz(scale, scale, scale)

# Show the plot to the screen
pyplot.show()